In [1]:
import random

In [2]:
def exp_mod(a: int, b: int, n: int) -> int:
    # Argumentos:
    # a: int - a >= 0
    # b: int - b >= 0
    # n: int - n > 0
    # Retorna:
    # int - a**b en modulo n
    if b == 0:
        return 1
    elif b%2 == 0:
        t = exp_mod(a, b//2, n)
        return (t*t)%n
    else:
        t = exp_mod(a, (b-1)//2, n)
        return (t*t*a)%n

In [3]:
def miller_rabin(n: int, k: int) -> bool:
    # Argumentos:
    # n: int - n >= 1
    # k: int - k >= 1
    # Retorna:
    # int - True si n es un numero primo, y False en caso contrario.
    # La probabilidad de error en el test debe ser menor o igual a 2**(-k)
    par = n - 1
    d = par // 2
#     print("d: ", d)
    exponente = 1
#     print("exponente: ", exponente)
    while d % 2 == 0:
        d = d // 2
#         print("d en el while: ", d)
        exponente += 1
#         print("exponente: ", exponente)
#     print("d: ", d)
#     print("exponente: ", exponente)
    for i in range(k):
        continuar = False
        rand_int = random.randint(2, n - 2)
        x = exp_mod(rand_int,d,n)
        if x == 1 or x == par:
            continuar = True
        if not continuar:
            for j in range(exponente - 1):
                x = exp_mod(x,2,n)
                if x == par:
                    continuar = True
                    break
        if not continuar:
            return False
    return True

In [4]:
def generar_primo(l: int) -> int:
    # Argumentos:
    # l: int - l >= 1
    # Retorna:
    # int - numero primo con al menos l digitos. La probabilidad de
    # error en la generacion debe ser menor o igual a 2**(-100)
    aleatorio = random.randint(10**(l-1), 10**(l))
    posible_primo = miller_rabin(aleatorio, 50)
    while not posible_primo:
        aleatorio = random.randint(10**(l-1), 10**(l))
        posible_primo = miller_rabin(aleatorio, 50)
    return aleatorio

In [5]:
def alg_ext_euclides(a: int, b: int) -> (int, int, int):
    # Argumentos :
    # a: int
    # b: int - a >= b 0 y a > 0
    # Retorna:
    # (int, int, int) - maximo comun divisor MCD(a, b) entre a y b,
    # numeros enteros s y t tales que MCD(a, b) = s*a + t*b
    s1 = 1
    t1 = 0
    s2 = 0
    t2 = 1
    resultado = a % b
    mcd = b
    while resultado != 0:
#         print("resultado: ", resultado)
#         print("a: ", a)
#         print("b: ", b)
        s_aux = s1 - (a // b)*s2
        t_aux = t1 - (a // b)*t2
        s1 = s2
        t1 = t2
        s2 = s_aux
        t2 = t_aux
#         print("s1: ", s1)
#         print("s2: ", s2)
#         print("t1: ", t1)
#         print("t2: ", t2)
        a = b
        b = resultado
        mcd = resultado
        resultado = a % b
    return (mcd, s2, t2)

In [6]:
def inverso(a: int, n: int) -> int:
    # Argumentos:
    # a: int - a >= 1
    # n: int - n >= 2, a y n son primos relativos
    # Retorna:
    # int - inverso de a en modulo n
    mcd, s, t = alg_ext_euclides(a,n)
    if mcd == 1:
        return s

In [29]:
def generar_clave(l: int):
    # Argumentos:
    # l: int - largo de las claves a ser generadas
    # Retorna:
    # genera una clave privada (d, N) y una clave publica (e, N) tales
    # que d, e y N tienen al menos l digitos. La clave privada debe
    # ser almacenada en un archivo private_key.txt en el formato:
    # d
    # N
    # La clave publica debe ser almacenada de la misma forma en
    # en un archivo public_key.txt
    P = generar_primo(20)
    Q = generar_primo(20)
    while P == Q:
        Q = generar_primo(20)
    N = P*Q
    phi_n = (P - 1)*(Q - 1)
    d = random.randint(0, phi_n - 1)
    mcd,s,t = alg_ext_euclides(phi_n, d)
    while mcd != 1:
        d = random.randint(0, phi_n - 1)
        mcd,s,t = alg_ext_euclides(phi_n, d)
    print("d: ", d)
    
    e = inverso(d, phi_n)
    
    with open("private_key.txt", "w") as private:
        private.write(str(d) +"\n" + str(N) + "\n")
    
    with open("public_key.txt", "w") as public:
        public.write(str(e) + "\n" + str(N) + "\n")

In [32]:
def enc(m: int) -> int:
    # Argumentos:
    # m: int - 0 <= m <= N-1, suponiendo que la clave publica
    # almacenada en public_key.txt es (e, N)
    # Retorna:
    # int: cifrado de m de acuerdo con la clave publica almacenada
    # en public_key.txt
    with open("public_key.txt") as public:
        primera = public.readline().strip()
        segunda = public.readline().strip()
    return exp_mod(m, int(primera), int(segunda))

In [34]:
def dec(m: int) -> int:
    # Argumentos:
    # m: int - 0 <= m <= N-1, suponiendo que la clave privada
    # almacenada en private_key.txt es (d, N)
    # Retorna:
    # int: descifrado de m de acuerdo con la clave privada
    # almacenada en private_key.txt
    with open("private_key.txt") as public:
        primera = public.readline().strip()
        segunda = public.readline().strip()
    return exp_mod(m, int(primera), int(segunda))

In [24]:
# miller_rabin(1, 50)

In [25]:
generar_primo(300)

198051923960035863321684561577783596098433697216555050232622077427846680049777814045154619869240299571996531787814173589073024149307103697108375728015861923313194164360629292014925136391753887853226631280261767219829691814685892602021920122402818404396786222982106612998868847372462253506199553933703

In [26]:
alg_ext_euclides(1001, 7)

(7, 0, 1)

In [27]:
exp_mod(10,8,10000000000000000000000000000)

100000000

In [53]:
inverso(86, 103)

6

In [31]:
generar_clave(20)

d:  1480376247924132563648525081489277184289


In [35]:
dec(100000)

1990360310074677706630628399489359784955